In [1]:
import numpy as np
import random

# Definimos el entorno
rows, cols = 3, 3  # Tamaño del entorno 3x3
actions = ['Arriba', 'Abajo', 'Izquierda', 'Derecha']  # Acciones posibles
num_states = rows * cols  # Número total de estados
rewards = np.zeros((rows, cols))  # Recompensas iniciales
rewards[2, 2] = 10  # Meta con recompensa
gamma = 0.9  # Factor de descuento
alpha = 0.5  # Tasa de aprendizaje
epsilon = 0.3  # Probabilidad inicial de exploración

In [2]:
# Inicializamos la Q-Table
q_table = np.zeros((num_states, len(actions)))

In [3]:
q_table

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [4]:
# Función para convertir estado 2D a índice 1D
def state_to_index(state):
    x, y = state
    return x * cols + y


# Función para convertir índice 1D a estado 2D
def index_to_state(index):
    return divmod(index, cols)

# Función para elegir una acción usando ε-Greedy
def choose_action(state_index, epsilon):
    if random.uniform(0, 1) < epsilon:  # Exploración
        return random.choice(range(len(actions)))
    else:  # Explotación
        return np.argmax(q_table[state_index])


# Función para tomar una acción y obtener el nuevo estado
def take_action(state, action):
    x, y = state
    if action == 0:  # Arriba
        if x > 0:
            x -= 1
        else:  # Penalización por salir del entorno
            return state, -1
    elif action == 1:  # Abajo
        if x < rows - 1:
            x += 1
        else:
            return state, -1
    elif action == 2:  # Izquierda
        if y > 0:
            y -= 1
        else:
            return state, -1
    elif action == 3:  # Derecha
        if y < cols - 1:
            y += 1
        else:
            return state, -1
    return (x, y), rewards[(x, y)]  # Retorna el nuevo estado y la recompensa

In [5]:
# Episodios de entrenamiento
episodes = 10
for episode in range(episodes):
    print(f"\nEpisodio {episode + 1}")
    state = (0, 0)  # Estado inicial
    state_index = state_to_index(state)
    done = False

    while not done:
        # Elegir acción
        action = choose_action(state_index, epsilon)
        next_state, reward = take_action(state, action)
        next_state_index = state_to_index(next_state)

        # Actualizar Q-Table usando la ecuación de Bellman
        max_q_next = np.max(q_table[next_state_index])
        q_table[state_index, action] += alpha * (reward + gamma * max_q_next - q_table[state_index, action])

        # Mostrar estado actual, acción y actualización
        print(f"Estado: {state}, Acción: {actions[action]}, Recompensa: {reward}")
        print(f"Q-Table actualizada (Estado {state_index}):\n{q_table}\n")

        # Si se alcanza la meta, termina el episodio
        if next_state == (2, 2):
            done = True

        # Mover al siguiente estado
        state, state_index = next_state, next_state_index

    # Reducir epsilon para explorar menos a medida que el agente aprende
    epsilon = max(0.1, epsilon * 0.9)



Episodio 1
Estado: (0, 0), Acción: Arriba, Recompensa: -1
Q-Table actualizada (Estado 0):
[[-0.5  0.   0.   0. ]
 [ 0.   0.   0.   0. ]
 [ 0.   0.   0.   0. ]
 [ 0.   0.   0.   0. ]
 [ 0.   0.   0.   0. ]
 [ 0.   0.   0.   0. ]
 [ 0.   0.   0.   0. ]
 [ 0.   0.   0.   0. ]
 [ 0.   0.   0.   0. ]]

Estado: (0, 0), Acción: Abajo, Recompensa: 0.0
Q-Table actualizada (Estado 0):
[[-0.5  0.   0.   0. ]
 [ 0.   0.   0.   0. ]
 [ 0.   0.   0.   0. ]
 [ 0.   0.   0.   0. ]
 [ 0.   0.   0.   0. ]
 [ 0.   0.   0.   0. ]
 [ 0.   0.   0.   0. ]
 [ 0.   0.   0.   0. ]
 [ 0.   0.   0.   0. ]]

Estado: (1, 0), Acción: Arriba, Recompensa: 0.0
Q-Table actualizada (Estado 3):
[[-0.5  0.   0.   0. ]
 [ 0.   0.   0.   0. ]
 [ 0.   0.   0.   0. ]
 [ 0.   0.   0.   0. ]
 [ 0.   0.   0.   0. ]
 [ 0.   0.   0.   0. ]
 [ 0.   0.   0.   0. ]
 [ 0.   0.   0.   0. ]
 [ 0.   0.   0.   0. ]]

Estado: (0, 0), Acción: Abajo, Recompensa: 0.0
Q-Table actualizada (Estado 0):
[[-0.5  0.   0.   0. ]
 [ 0.   0.   0.   0. 

In [6]:
# Mostrar la Q-Table final
print("\nQ-Table final:")
print(q_table)


Q-Table final:
[[-0.984375    6.04059082 -0.31802344  0.        ]
 [-0.984375    3.290625    0.          0.        ]
 [-0.984375    0.          0.         -0.99609375]
 [ 0.20503125  0.          0.14101563  7.65703125]
 [ 0.          0.          0.          8.90332031]
 [ 0.          9.99023438  0.          3.4453125 ]
 [ 0.          0.          0.          0.        ]
 [ 0.         -0.5         0.          0.        ]
 [ 0.          0.          0.          0.        ]]


# Inferencia

In [7]:
# Función para ejecutar la inferencia
def run_inference(q_table, start_state, goal_state):
    state = start_state
    state_index = state_to_index(state)
    path = [state]  # Para registrar el camino recorrido

    print("\nInferencia:")
    while state != goal_state:
        # Seleccionar la acción óptima
        action = np.argmax(q_table[state_index])
        print(f"Estado: {state}, Acción elegida: {actions[action]}")

        # Moverse al siguiente estado
        next_state, reward = take_action(state, action)
        next_state_index = state_to_index(next_state)

        # Registrar el camino
        path.append(next_state)

        # Actualizar el estado actual
        state, state_index = next_state, next_state_index

    print("\nCamino recorrido:")
    print(path)

# Ejecutar inferencia desde un estado inicial (0, 0) hasta la meta (2, 2)
start_state = (0, 0)
goal_state = (2, 2)
run_inference(q_table, start_state, goal_state)



Inferencia:
Estado: (0, 0), Acción elegida: Abajo
Estado: (1, 0), Acción elegida: Derecha
Estado: (1, 1), Acción elegida: Derecha
Estado: (1, 2), Acción elegida: Abajo

Camino recorrido:
[(0, 0), (1, 0), (1, 1), (1, 2), (2, 2)]
